In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from tqdm import tqdm
from torchsummary import summary

from concrete import fhe
from sklearn.model_selection import train_test_split
import time

/home/local/ASURITE/nnjungle/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#hyperparams
no_epochs = 2
batch_size = 32
learning_rate = 0.001

In [4]:
#transform the dataset into tensors normalized range [-1, 1]
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5),(0.5))     
        ])

In [5]:
#data sets downloading and reading
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform
                                        )

test_dataset = torchvision.datasets.MNIST(root='data',
                                        train=False,
                                        download=True,
                                        transform=transform
                                        )


In [6]:
# Extract features (images) and labels from MNIST dataset
mnist_features = train_dataset.data.numpy().reshape(-1, 28, 28)
mnist_labels = train_dataset.targets.numpy()

# Reshape and expand dimensions to match the structure of load_digits dataset
x_train_mnist = np.expand_dims(mnist_features, 1)

# Split the MNIST data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(
    x_train_mnist, mnist_labels,  train_size=100, test_size=100, shuffle=True, random_state=42
)

#x_train = x_train.astype('float64')
print(x_train.dtype)
print(x_train.shape)

#print(x_train)
# plt.imshow(x_train[0,0], cmap='grey')
# plt.show()
# Verify the shapes
print("Shape of x_train from MNIST:", x_train.shape)
print("Shape of x_test from MNIST:", x_test.shape)
print("Shape of y_train from MNIST:", y_train.shape)
print("Shape of y_test from MNIST:", y_test.shape)

uint8
(100, 1, 28, 28)
Shape of x_train from MNIST: (100, 1, 28, 28)
Shape of x_test from MNIST: (100, 1, 28, 28)
Shape of y_train from MNIST: (100,)
Shape of y_test from MNIST: (100,)


In [7]:
configuration = fhe.Configuration(
    comparison_strategy_preference=fhe.ComparisonStrategy.ONE_TLU_PROMOTED,
)


Create the lookup table to used for the spikes

In [8]:
# array_table = np.zeros(1000)
# array_table[:500] = 1
lookup_table = (0,) * 500 + (1,) * 500

# lookup_table = [array_table]
# print(lookup_table)

table = fhe.LookupTable(lookup_table)

In [21]:
# weight = np.array([[2, 1], [3, 2]]).reshape(1, 1, 2, 2)
# selected_weights = np.array(x_train[0, 0, :, :]).reshape(1, 1, 28, 28)
weight = np.random.randint(0, 4, size=(1, 1, 3, 3))
print(weight.shape)
fhe_spikes_array = np.empty(shape=(1, 1, 1, 24), dtype=object)
# print(weight)

@fhe.compiler({"x": "encrypted"})
def network_function(x):
    print("we dey here")
    x = fhe.conv(x, weight, strides=(1, 1))
    print("first conv done")
    x = fhe.conv(x, weight, strides=(1, 1))
    print("second conv done")
    # x = fhe.maxpool(x, kernel_shape=(2, 2), strides=(2, 2), dilations=(1, 1))
    print("maxpool done")
    x = pbs_function(x)
    print("pbs_func done")
    # conv2 = fhe.conv(conv1, selected_weights)
    # print(pool1)
    net_result = x
    return net_result

def pbs_function(xpbs):
    print("in the pbs")
    # spiking_array = table[xpbs[0,0,0,1]]
    print(xpbs[0])
    # return spiking_array
    # spiking_array = fhe.array(xpbs)
    for i, x_pbs_row in enumerate(xpbs):
        for j, pbs_row in enumerate(x_pbs_row):
            for k, row in enumerate(pbs_row):
                for l, cell in enumerate(row):
                    xpbs[i, j, k, l] = table[cell]
                    print(f'array[{i,j,k,l}]')
                    # return fhe_spikes_array
    return xpbs

(1, 1, 3, 3)


In [22]:
x_train = np.array(x_train[0, 0, :, :]).reshape(1, 1, 28, 28)
print(x_train.shape)

inputset = [np.random.randint(0, 4, size=(1, 1, 28, 28)) for _ in range(10)]
# print(inputset)
circuit = network_function.compile(inputset)
start = time.time()
enc_sampe = circuit.encrypt(x_train)
enc_result = circuit.run(enc_sampe)
dec_result = circuit.decrypt(enc_result)
print(dec_result.shape)
print(dec_result)
end = time.time()

print(f'total time {end - start:.2f}s')

(1, 1, 28, 28)
we dey here
first conv done
second conv done
maxpool done
in the pbs
Tracer<output=EncryptedTensor<uint2, shape=(1, 24, 24)>>
pbs_func done


: 